In [1]:
import pandas as pd
import numpy as np
import ast
import os
import cv2
from os import getcwd

## Setup

In [2]:
# File with object annotations
da = pd.read_csv('DaycaresAnnotations_original.csv')

# Filepaths to folders
filepath_to_daycares = 'Daycares/'
filepath_to_nondaycares = 'NonDaycares/'
filepath_to_labels = 'labels/'

# Name of file that will have annotations
save_to_file = 'annotate.txt'
image_path_file = 'train.txt'

## Parse

In [3]:
# Annotated output
output_annot = []
image_paths = []
cwd = os.getcwd()
###########################################################################
# DAYCARES
for r in range(len(da)):
    # Check if it has an object
    if da.loc[r, 'region_shape_attributes'] != "{}":
        # filename
        filepath = filepath_to_daycares + da.loc[r, 'filename']
        imagepath = cwd +'/'+ filepath
        filepath_label = filepath_to_daycares + da.loc[r, 'filename'] # both jpg and text need to be in same dir
#         filepath_label = filepath_to_labels + da.loc[r, 'filename']
        filepath_label = filepath_label.replace('.jpg', '.txt')
        try:
            img = cv2.imread(filepath)

            # Get rect attributes
            rect = ast.literal_eval(da.loc[r, 'region_shape_attributes'])

            height, width, channels = img.shape
            rel_x = rect['x'] / width
            rel_y = rect['y'] / height
            rel_width = rect['width'] / width
            rel_height = rect['height'] / height

            # print(height, width, channels)
            # class_name ("0" = Daycare) as we have only 1 class

            current_annot = '{} {} {} {} {}'.format(0, rel_x, rel_y, rel_width, rel_height)
#             current_annot += "1"
            image_paths.append(imagepath)
            output_annot.append(current_annot)
            with open(filepath_label, 'w') as f:
                f.write("%s\n" % current_annot)
    
        except FileNotFoundError:
            print(filepath, ' not found')



In [ ]:
###########################################################################
# Non DAYCARES
nod = os.listdir(filepath_to_nondaycares)
for nd in nod:
    if nd.endswith('.jpg'):
        # filename
        current_annot = filepath_to_nondaycares + nd + ","
        
        # x-min (Does not matter for NonDaycare)
        current_annot += str(0) + ","
        # x-max (Does not matter for NonDaycare)
        current_annot += str(0) + ","
        # y-min (Does not matter for NonDaycare)
        current_annot += str(0) + ","
        # y-max (Does not matter for NonDaycare)
        current_annot += str(0) + ","
        
        # class_name ("1" = Daycare, "0" = NonDaycare)
        current_annot += "0"
        
        output_annot.append(current_annot)


In [4]:
# Remove duplicates
output_annot = list(set(output_annot))
image_paths = list(set(image_paths))

## Check output

In [5]:
len(output_annot)

815

In [6]:
output_annot

['0 0.3510992671552298 0.7235169491525424 0.06862091938707528 0.0815677966101695',
 '0 0.48567621585609594 0.3919491525423729 0.05263157894736842 0.07944915254237288',
 '0 0.5123251165889408 0.4936440677966102 0.06595602931379081 0.08050847457627118',
 '0 0.42504996668887407 0.5572033898305084 0.04330446369087275 0.09216101694915255',
 '0 0.03530979347101932 0.2807203389830508 0.057961359093937376 0.09110169491525423',
 '0 0.508327781479014 0.576271186440678 0.0899400399733511 0.11546610169491525',
 '0 0.502998001332445 0.3485169491525424 0.0626249167221852 0.0815677966101695',
 '0 0.34710193204530315 0.7192796610169492 0.06862091938707528 0.0815677966101695',
 '0 0.6002664890073285 0.7680084745762712 0.051299133910726186 0.1059322033898305',
 '0 0.4990006662225183 0.5328389830508474 0.04930046635576282 0.1016949152542373',
 '0 0.3331112591605596 0.2807203389830508 0.056628914057295136 0.13559322033898305',
 '0 0.45436375749500335 0.4300847457627119 0.04463690872751499 0.07415254237288

In [7]:
print(len(image_paths))
image_paths

2494


['/Users/muhammadmuhaimin/SFU/Big-Data/880-Deep-Learning/Satellite-Daycare-detection/data/Daycares/1909_20160311.jpg',
 '/Users/muhammadmuhaimin/SFU/Big-Data/880-Deep-Learning/Satellite-Daycare-detection/data/Daycares/1903_20150813.jpg',
 '/Users/muhammadmuhaimin/SFU/Big-Data/880-Deep-Learning/Satellite-Daycare-detection/data/Daycares/19171_20140506.jpg',
 '/Users/muhammadmuhaimin/SFU/Big-Data/880-Deep-Learning/Satellite-Daycare-detection/data/Daycares/13690_20120919.jpg',
 '/Users/muhammadmuhaimin/SFU/Big-Data/880-Deep-Learning/Satellite-Daycare-detection/data/Daycares/1479_20160807.jpg',
 '/Users/muhammadmuhaimin/SFU/Big-Data/880-Deep-Learning/Satellite-Daycare-detection/data/Daycares/1752_20140404.jpg',
 '/Users/muhammadmuhaimin/SFU/Big-Data/880-Deep-Learning/Satellite-Daycare-detection/data/Daycares/17471_20170218.jpg',
 '/Users/muhammadmuhaimin/SFU/Big-Data/880-Deep-Learning/Satellite-Daycare-detection/data/Daycares/1913_20120625.jpg',
 '/Users/muhammadmuhaimin/SFU/Big-Data/880-De

## Save to file

In [ ]:
# Save to FILE
with open(save_to_file, 'w') as f:
    for item in output_annot:
        f.write("%s\n" % item)

In [8]:
# Save image paths to FILE
with open(image_path_file, 'w') as f:
    for item in image_paths:
        f.write("%s\n" % item)